In [32]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils import shuffle
import torch.nn.functional as F

In [ ]:
# 加载数据集
data = pd.read_csv('dataset.csv')

# 随机排序数据集
data = shuffle(data)

# 划分特征和标签
X = data[['data1', 'data2']].values
y = data['label'].values
y = y-1

#数据集分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
y_test = torch.tensor(y_test,dtype=torch.long)

y_train = F.one_hot(y_train)
y_test = F.one_hot(y_test)
_, y_test_labels = torch.max(y_test, 1)

# 创建TensorDataset和DataLoader用于批量训练
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [34]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out


# 初始化模型
model = MLP()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9, lr=0.001)


In [35]:
num_epochs = 150
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets.float())
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 1 == 0:
        with torch.no_grad():
            outputs = model(X_test)
            loss1 = criterion(outputs, y_test.float())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_test_labels).sum().item() / y_test.size(0)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train_Loss: {loss.item():.4f}, Test_Loss: {loss1.item():.4f}, Test Accuracy: {accuracy:.2f}')

Epoch [1/150], Train_Loss: 1.3495, Test_Loss: 1.3639, Test Accuracy: 0.53
Epoch [2/150], Train_Loss: 1.3254, Test_Loss: 1.3396, Test Accuracy: 0.68
Epoch [3/150], Train_Loss: 1.3379, Test_Loss: 1.3119, Test Accuracy: 0.62
Epoch [4/150], Train_Loss: 1.2342, Test_Loss: 1.2811, Test Accuracy: 0.55
Epoch [5/150], Train_Loss: 1.2412, Test_Loss: 1.2499, Test Accuracy: 0.52
Epoch [6/150], Train_Loss: 1.2413, Test_Loss: 1.2221, Test Accuracy: 0.52
Epoch [7/150], Train_Loss: 1.2438, Test_Loss: 1.1992, Test Accuracy: 0.54
Epoch [8/150], Train_Loss: 1.2331, Test_Loss: 1.1811, Test Accuracy: 0.57
Epoch [9/150], Train_Loss: 1.1196, Test_Loss: 1.1670, Test Accuracy: 0.61
Epoch [10/150], Train_Loss: 1.0633, Test_Loss: 1.1558, Test Accuracy: 0.64
Epoch [11/150], Train_Loss: 1.1669, Test_Loss: 1.1466, Test Accuracy: 0.68
Epoch [12/150], Train_Loss: 1.0735, Test_Loss: 1.1386, Test Accuracy: 0.70
Epoch [13/150], Train_Loss: 1.0956, Test_Loss: 1.1313, Test Accuracy: 0.71
Epoch [14/150], Train_Loss: 1.1730